In [ ]:
%matplotlib inline

In [ ]:
%env SHELL=/bin/bash

In [ ]:
import matplotlib.pylab as plt
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12, 10),
         'axes.labelsize': 13,
         'axes.titlesize':13,
         'xtick.labelsize':13,
         'ytick.labelsize':13,
         'axes.labelweight':'bold',
          'legend.fontsize': 13,
          'font.size':13,
         'figure.max_open_warning': 0}

plt.rcParams.update(params)
import matplotlib.colors as colors
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, kurtosis, skew

In [ ]:
import dask.array as da
from daskms import xds_from_table, xds_to_table
from dask.distributed import Client

import numpy as np

In [ ]:
client = Client(n_workers=4, 
                threads_per_worker=4, 
                processes=True, 
                memory_limit='8GB'
                )
client

# Setting the data path

In [ ]:
data_path = '/home/relebohile/Desktop/OT_ASSIGNMENT/'

# Create xarray datasets from Measurement Set 

In [ ]:
ds = xds_from_table(data_path+"1548939342.ms")

In [ ]:
print(ds)

# Getting the keys in the MS

In [ ]:
for key in ds[0].keys():
    print(key)

# Some EDA

In [ ]:
obs_table = xds_from_table(data_path+"1548939342.ms/::OBSERVATION")
print(obs_table)

In [ ]:
# obs = xds_from_table(data_path+"1548939342.ms/:OBSERVATION_ID")

obs = ds[0]["FIELD_ID"]

In [ ]:
for key in obs_table[0].keys():
    print(f'{key}: {obs_table[0][key].values}')

In [ ]:
A1 = ds[0]["ANTENNA1"]
A2 = ds[0]["ANTENNA2"]
ant_list = list(set(np.append(A1,A2)))
nants = len(ant_list)
inter = ds[0]["INTERVAL"]
print(f'Total number of antennas used {nants}')
print('Interval Table Shape: ',inter.shape)
print(f'Dump rate is {np.median(inter)} s');

In [ ]:
A1 = ds[0]["ANTENNA1"]

In [ ]:
A1

# Understanding the UV coverage

In [ ]:
# Get the UVW points
ds[0]['UVW']

In [ ]:
u,v,w = ds[0]['UVW'][:,0],ds[0]['UVW'][:,1],ds[0]['UVW'][:,2]

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(u,v,'.')
plt.xlabel('u')
plt.ylabel('v')

We observe shorter spacing closer to the uv origin, and there are more points in that region, reflecting a good coverage on the uv origin, as we move away from the origin the spacing increases. We also see that there are regions away from the uv origin were there's some good coverage, and this appears as densed points.

In [ ]:

# Read the ANTENNA table
antenna_table = xds_from_table(data_path+"1548939342.ms::ANTENNA")

print(antenna_table)

In [ ]:
antenna_table[0]

In [ ]:
antenna_table[0]['NAME'].values

In [ ]:
antenna_table[0]['DISH_DIAMETER'].values

In [ ]:
antenna_table[0]['POSITION']

In [ ]:
print(antenna_table[0]['POSITION'].values)

In [ ]:
X,Y,Z = antenna_table[0]['POSITION'][:,0],antenna_table[0]['POSITION'][:,1],antenna_table[0]['POSITION'][:,2]

In [ ]:
plt.figure(figsize=(6,5))
plt.plot(X,Y, 'o')

In [ ]:
ls

In [ ]:
# Read the SPECTRAL table
spectral_table = xds_from_table(data_path+"1548939342.ms::SPECTRAL_WINDOW")

In [ ]:
spectral_table[0]

In [ ]:
frequencies = spectral_table[0]['CHAN_FREQ'].values
nchannels = spectral_table[0]['NUM_CHAN'].values[0]
channel_width = spectral_table[0]['CHAN_WIDTH'].values[0][0]
REF_FREQ = spectral_table[0]['REF_FREQUENCY'].values[0]


summary = {
    'Frequencies': frequencies,
    'No_Channeles': nchannels,
    'Channel_Width': channel_width,
    }

In [ ]:
print(f"Frequency range (MHz): {summary['Frequencies'].min()/1.e6} - {summary['Frequencies'].max()/1.e6}")  
print(f"Total no of channles {summary['No_Channeles']}")
print(f"Channel width (kHz): {channel_width/1.e3}")

# Working with Visibilities

In [ ]:
# Let us now try to get the visibilities
vis = ds[0]['DATA']

In [ ]:
plt.figure(figsize=(6,5))
plt.imshow(np.abs(vis[:,:,0]).T, 
           aspect= 'auto', 
           origin='lower', 
           cmap='PuOr', 
           norm = colors.LogNorm()
           );
plt.ylabel('Channel No')
plt.xlabel('Time')
plt.colorbar();

In [ ]:
plt.figure(figsize=(6,5))
plt.imshow(np.abs(vis[:,:,1]).T, 
           aspect= 'auto', 
           origin='lower', 
           cmap='PuOr', 
           norm = colors.LogNorm()
           );
plt.ylabel('Channel No')
plt.xlabel('Time')
plt.colorbar();

In [ ]:
# Doing a 1-D cut across the visibility 
plt.figure(figsize=(6,5))
plt.plot(np.abs(vis[500,:,0]))
plt.semilogy()
plt.ylabel('Amplitude')
plt.xlabel('Channels')

In [ ]:
phaset2selected = np.angle(vis[500,:,0].T)

In [ ]:
plt.figure(figsize=(6,5))
plt.plot(phaset2selected)
plt.ylabel('Phase')
plt.xlabel('Channel')

In [ ]:
plt.figure(figsize=(6,5))
plt.plot(np.unwrap(phaset2selected))
plt.ylabel('Phase')
plt.xlabel('Channel')

In [ ]:
def Gauss(x, dc, a, x0, sigma):
    '''
    This will fit a gaussian function to a data set
    '''
    return dc+a * np.exp(-(x - x0)**2 / (2 * sigma**2))

In [ ]:
#
# Plot the average across frequency slice
#
frequencies = spectral_table[0]['CHAN_FREQ'].values.T
absdata = np.abs(vis[500,:,0]).compute()

fig = plt.figure(figsize=(6,5))
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()

ax1.plot(frequencies/1.e6,np.log10(absdata),'-')
ax1.set_xticks(np.arange(frequencies[0]/1.e6,frequencies[-1]/1.e6, step=100))
ax1.set_xlabel('Frequency [MHz]')
ax1.set_ylabel('Amplitude')
ax2.plot(np.log10(absdata))
ax2.set_xticks(np.arange(0,absdata.shape[0], step=256))
ax2.set_xlabel('Index no')
##pair 1
ax1.axvline(1070, color='red', linestyle='--', linewidth=0.5, label='RFI Threshold (850 MHz)')
ax1.axvline(1520, color='red', linestyle='--', linewidth=0.5, label='RFI Threshold (850 MHz)')
plt.grid()

## Define 3 regions where you notice RFI and explain what are the major sources of RFI in these regions.

#### RFI was observed to appear in three regions out of the four plotted regions above, 
#####
#### We observe the RFI in region1, Frequency (856-960 MHz) which is within index no1-256, this is indicated by sudden spikes in the signal. In this region RFI could be possibly as a result of Mobile phone towers and microwave links. Mobile communication towers emit signals in this range, leading to interference in radio astronomy observations. Signals from nearby towers can create strong, localized interference.
#####
#### We observe RFI again in region2 Frequency (1070-1285 MHz) which is within the index no256-512, here we also see sharp spikes. This could be caused by Communication Satellites and Television Broadcasting. The presence of strong, continuous signals from commercial and governmental satellite communication, alongside other terrestrial communication sources, makes this range particularly susceptible to RFI
####

#### From index number 512-768 we see a clean region, where there is no any interference,
####
#### Laslty we see some RFI from index number 768, where the frequency is above 1520MHz. This could be caused by Mobile Communication Systems and Satellite Communication. The high density of communication signals from various systems, including mobile networks and satellite communications, creates a noisy environment in this range.

In [ ]:
from scipy.optimize import curve_fit, minimize_scalar

In [ ]:
# 350 - 400 good spike
# Use 50-150 for 1548939342.ms
ini_idx = int(input('Enter initial index for Gaussian fit '))
fin_idx = int(input('Enter final index for Gaussian fit '))

x = np.ravel(frequencies[ini_idx:fin_idx])
y = np.log10(absdata[ini_idx:fin_idx])

# weighted arithmetic mean (corrected - check the section below)
mean = (sum(x * y) / sum(y)).values
sigma = (np.sqrt(sum(y * (x - mean)**2) / sum(y))).values

dc = np.nanmean(np.log10(absdata[550:750]).values)-1

popt,pcov = curve_fit(Gauss, x, y, p0=[max(y), dc, mean, sigma])

plt.plot(x/1e6, y, 'b+:', label='data')
x_new = np.arange(frequencies[ini_idx], frequencies[fin_idx],15)
plt.plot(x_new/1e6, Gauss(x_new, *popt), 'r-', label='fit')
plt.legend()

plt.ylabel('Amplitude [dB]')
plt.xlabel('Frequency [MHz]')
plt.show()

print (f"FWHM [MHz] =  {(2.*np.sqrt(2.*np.log(2.))*sigma)/1.e6}")
print(f"Maximum amplitude is at [db], {max(Gauss(x_new, *popt))} ")
print (f"Maximum is at [MHz]  {x_new[np.argmax(Gauss(x_new, *popt))]/1.e6}")
print(f"Number of channels affected {int(((2.*np.sqrt(2.*np.log(2.))*sigma))/(channel_width))}")

## 2. What can you infer from the histogram distribution of the RFI signal and a ‘clean’ region?

The histogram distribution of RFI signals compared to flat ‘clean’ regions provides important insights into the data. RFI typically shows distinct peaks, indicating strong interference, while the clean regions appear flat, suggesting a consistent background noise level with minimal variability. The presence of pronounced peaks in the RFI distribution highlights interference that may obscure genuine astrophysical signals. By comparing the peak values of RFI to the flat noise levels in the clean regions, the Signal-to-Noise Ratio (SNR) can be assessed; higher peaks in the RFI indicate stronger interference. Additionally, the flat nature of the clean regions can inform threshold settings for RFI mitigation, enabling effective filtering of interference while preserving the underlying signal. Changes in the RFI histogram over time or location may also reveal specific patterns or sources of interference.

In [ ]:
# # Generate fake data
# # rng = np.random.default_rng(0)
# # x = np.linspace(-5., 5., 200)
# # y = 3 * np.exp(-0.5 * (x - 1.3)**2 / 0.8**2)
# # y += rng.normal(0., 0.2, x.shape)

# # Fit the data using a box model.
# # Bounds are not really needed but included here to demonstrate usage.


# # Fit the data using a Gaussian
# g_init = models.Gaussian1D(amplitude=1., mean=0, stddev=1.)
# fit_g = fitting.LevMarLSQFitter()
# g = fit_g(g_init, x, y)

# # Plot the data with the best-fit model
# plt.figure(figsize=(8,5))
# plt.plot(x, y, 'k-+')
# plt.plot(x, g(x), label='Gaussian')
# plt.xlabel('Position')
# plt.ylabel('Flux')
# plt.legend(loc=2)

In [ ]:
vis_clean = []

vis_rfi = []

for a1 in range(0,10):

   for a2 in range(1,10):

      # print(antenna_table[0].NAME.data[a1].compute(), antenna_table[0].NAME.data[a2].compute())

      idx = np.where((ds[0].ANTENNA1 == a1) & (ds[0].ANTENNA2 == a2))[0]

      # antenna_table[0].NAME.data[a1].compute(), antenna_table[0].NAME.data[a2].compute()

   vis_rfi.append(np.abs(ds[0].DATA.data[idx][:,400,0].compute()))

   vis_clean.append(np.abs(ds[0].DATA.data[idx][:,600,0].compute()))

combined_array_clean = np.concatenate(vis_clean)

combined_array_rfi = np.concatenate(vis_rfi)

np.mean(combined_array_clean), np.mean(combined_array_rfi)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, kurtosis, skew

# Assuming combined_array_clean and combined_array_rfi are already defined

# Slice the data to include only values between 0 and 6
clean_slice = combined_array_clean[(combined_array_clean >= 0) & (combined_array_clean <= 6)]
rfi_slice = combined_array_rfi[(combined_array_rfi >= 0) & (combined_array_rfi <= 6)]

# Compute kurtosis and skewness for both clean and RFI slices
kurtosis_clean = kurtosis(clean_slice)
skewness_clean = skew(clean_slice)

kurtosis_rfi = kurtosis(rfi_slice)
skewness_rfi = skew(rfi_slice)

# Print kurtosis and skewness values
print(f"Clean signal : kurtosis = {kurtosis_clean:.2f}, skewness = {skewness_clean:.2f}")
print(f"RFI signal: kurtosis = {kurtosis_rfi:.2f}, skewness = {skewness_rfi:.2f}")

# Plot histograms for sliced data
plt.hist(combined_array_clean, bins=500, histtype=u'step', lw=2, density=True, label="Clean")
plt.hist(combined_array_rfi, bins=500, histtype=u'step', lw=2, density=True, label="RFI")

# Fit Gaussian to the sliced 'clean' array
mu_clean, std_clean = norm.fit(clean_slice)

# Fit Gaussian to the sliced 'RFI' array
mu_rfi, std_rfi = norm.fit(rfi_slice)

# Create x values for plotting the Gaussian
x = np.linspace(0, 6, 1000)  # Adjust x values to match the slicing range

# Plot Gaussian for 'clean' array
p_clean = norm.pdf(x, mu_clean, std_clean)
plt.plot(x, p_clean, 'r--', lw=2, label=rf'Gaussian Fit Clean: $\mu$={mu_clean:.2f}, $\sigma$={std_clean:.2f}')

# Plot Gaussian for 'RFI' array
p_rfi = norm.pdf(x, mu_rfi, std_rfi)
plt.plot(x, p_rfi, 'b--', lw=2, label=rf'Gaussian Fit RFI: $\mu$={mu_rfi:.2f}, $\sigma$={std_rfi:.2f}')

# Add labels and legends
plt.xlabel("Amplitude Bin")
plt.ylabel("Counts")
plt.legend(fontsize=13)
plt.xlim(0, 6)  # Adjust x-axis limits to match the slicing range
plt.savefig("Gaussian_fit.png")
# Show plot
plt.show()

# Print the Gaussian fit parameters
print(f"Clean signal Gaussian fit: mean = {mu_clean:.2f}, std = {std_clean:.2f}")
print(f"RFI signal Gaussian fit : mean = {mu_rfi:.2f}, std = {std_rfi:.2f}")

### 5) Select a short baseline and a long baseline visibilities and explain what you notice in the contaminated region.

In [ ]:
vis_clean1 = []

vis_rfi1 = []

for a1_ in range(0,10):

   for a2_ in range(30,40):

      # print(antenna_table[0].NAME.data[a1].compute(), antenna_table[0].NAME.data[a2].compute())

      idx = np.where((ds[0].ANTENNA1 == a1_) & (ds[0].ANTENNA2 == a2_))[0]

      # antenna_table[0].NAME.data[a1].compute(), antenna_table[0].NAME.data[a2].compute()

   vis_rfi1.append(np.abs(ds[0].DATA.data[idx][:,400,0].compute()))

   vis_clean1.append(np.abs(ds[0].DATA.data[idx][:,600,0].compute()))

combined_array_clean1 = np.concatenate(vis_clean1)

combined_array_rfi1 = np.concatenate(vis_rfi1)

In [ ]:
# plt.hist(combined_array_clean1, bins=1500, histtype=u'step', lw=2, density=True, label="Clean")
# plt.hist(combined_array_rfi1, bins=1500, histtype=u'step', lw=2, density=True, label="RFI")

In [ ]:
# Assuming combined_array_clean and combined_array_rfi are already defined

# Slice the data to include only values between 0 and 6
combined_array_clean = combined_array_clean1
combined_array_rfi = combined_array_rfi1
clean_slice = combined_array_clean[(combined_array_clean >= 0) & (combined_array_clean <= 6)]
rfi_slice = combined_array_rfi[(combined_array_rfi >= 0) & (combined_array_rfi <= 6)]

# Compute kurtosis and skewness for both clean and RFI slices
kurtosis_clean = kurtosis(clean_slice)
skewness_clean = skew(clean_slice)

kurtosis_rfi = kurtosis(rfi_slice)
skewness_rfi = skew(rfi_slice)

# Print kurtosis and skewness values
print(f"Clean signal : kurtosis = {kurtosis_clean:.2f}, skewness = {skewness_clean:.2f}")
print(f"RFI signal: kurtosis = {kurtosis_rfi:.2f}, skewness = {skewness_rfi:.2f}")

# Plot histograms for sliced data
plt.hist(combined_array_clean, bins=1500, histtype=u'step', lw=2, density=True, label="Clean")
plt.hist(combined_array_rfi, bins=1500, histtype=u'step', lw=2, density=True, label="RFI")

# Fit Gaussian to the sliced 'clean' array
mu_clean, std_clean = norm.fit(clean_slice)

# Fit Gaussian to the sliced 'RFI' array
mu_rfi, std_rfi = norm.fit(rfi_slice)

# Create x values for plotting the Gaussian
x = np.linspace(0, 6, 1000)  # Adjust x values to match the slicing range

# Plot Gaussian for 'clean' array
p_clean = norm.pdf(x, mu_clean, std_clean)
#plt.plot(x, p_clean, 'r--', lw=2, label=rf'Gaussian Fit Clean: $\mu$={mu_clean:.2f}, $\sigma$={std_clean:.2f}')

# Plot Gaussian for 'RFI' array
p_rfi = norm.pdf(x, mu_rfi, std_rfi)
#plt.plot(x, p_rfi, 'b--', lw=2, label=rf'Gaussian Fit RFI: $\mu$={mu_rfi:.2f}, $\sigma$={std_rfi:.2f}')

# Add labels and legends
plt.xlabel("Amplitude Bin")
plt.ylabel("Counts")
plt.legend(fontsize=13)
plt.savefig("Gaussian_fit.png")
# Show plot
plt.show()

# Print the Gaussian fit parameters
print(f"Clean signal Gaussian fit: mean = {mu_clean:.2f}, std = {std_clean:.2f}")
print(f"RFI signal Gaussian fit : mean = {mu_rfi:.2f}, std = {std_rfi:.2f}")

### 6) Extract the HPBW of the RFI at 1500 MHz

In [ ]:
rfi_amplitude = combined_array_rfi 

# Step 1: Find the peak of the RFI signal
peak_amplitude = np.max(rfi_amplitude)

# Step 2: Calculate the half-power point (half of the peak)
half_power_value = peak_amplitude / 2

# Step 3: Find the points where the amplitude crosses the half-power value
indices = np.where(rfi_amplitude >= half_power_value)[0]

# Step 4: HPBW is the difference between the first and last indices where the half-power is met
hpbw = indices[-1] - indices[0]

# Convert this to appropriate units if necessary (e.g., MHz or degrees depending on the data scale)
print(f"HPBW of the RFI at 1.5 MHz: {hpbw}")

In [ ]:
frequencies = spectral_table[0]['CHAN_FREQ'].values
absdata = np.abs(vis[500,:,0]).compute()

Freq = frequencies[0][771:790]
Amp = absdata[771:790].values

In [ ]:
rfi_amplitude = Amp

# Step 1: Find the peak of the RFI signal
peak_amplitude = np.max(rfi_amplitude)

# Step 2: Calculate the half-power point (half of the peak)
half_power_value = peak_amplitude / 2

# Step 3: Find the points where the amplitude crosses the half-power value
indices = np.where(rfi_amplitude >= half_power_value)[0]

# Step 4: HPBW is the difference between the first and last indices where the half-power is met
hpbw = indices[-1] - indices[0]

print(f"HPBW of the RFI at 1.5 MHz: {hpbw}")

In [ ]:
times = ds[0]['TIME'].values

start_time_mjd = times.min() / 86400.0
end_time_mjd = times.max() / 86400.0

#Print the start and end times
print("Start time (MJD):", start_time_mjd)
print("End time (MJD):", end_time_mjd)
print("-----------------------------------------------")

Field_table = xds_from_table(data_path+"1548939342.ms/::FIELD")

for key in Field_table[0].keys():
    if key == "NAME":
        print(f'The target name is: {key}: {Field_table[0][key].values}')
print("-----------------------------------------------")

for key in spectral_table[0].keys():
    if key == "TOTAL_BANDWIDTH":
        print(f'{key}: {spectral_table[0][key].values}')

### Understanding the UVW

The UVW coordinates represent the spatial frequency plane, where U and V correspond to the baseline lengths in the East-West and North-South directions, respectively, and W is the coordinate along the line of sight. In contrast, the ENU coordinate system is tied to a specific terrestrial location, with East, North, and Up directions defining the local horizontal plane. To bridge these two systems, radio interferometry employs a transformation that accounts for the observer's geographic position and the Earth's curvature, enabling the conversion of baseline measurements into a form that can be directly related to astronomical images. This transformation is essential for accurate imaging and analysis of celestial objects, as it allows astronomers to interpret the observed data in terms of their angular positions on the sky.

$$
\begin{bmatrix}
U \\
V \\
W
\end{bmatrix}
=
\begin{bmatrix}
\cos(\alpha) & -\sin(\alpha) & 0 \\
\sin(\alpha) & \cos(\alpha) & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
E \\
N \\
U
\end{bmatrix}
$$

where ($\alpha$) is the azimuth angle defined by the observer's position.

### 2) Which antenna pairs make the shortest and longest baseline (give the antenna names)

In [ ]:
X = antenna_table[0]['POSITION'][:, 0].values  # x-coordinates
Y = antenna_table[0]['POSITION'][:, 1].values  # y-coordinates
Z = antenna_table[0]['POSITION'][:, 2].values  # z-coordinates (ensure you use the correct index)
ANTENNA_NAME = antenna_table[0]['NAME'].values  # antenna names

antennas = {name: (x, y, z) for name, x, y, z in zip(ANTENNA_NAME, X, Y, Z)}

shortest_baseline = float('inf')
longest_baseline = 0
shortest_pair = None
longest_pair = None

# Calculating the distances between each pair of antennas
for name1, (x1, y1, z1) in antennas.items():
    for name2, (x2, y2, z2) in antennas.items():
        if name1 != name2:  # Ensuring that I don't compare the same antenna
            distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)  # Using the 3D distance formula
            if distance < shortest_baseline:
                shortest_baseline = distance
                shortest_pair = (name1, name2)
            if distance > longest_baseline:
                longest_baseline = distance
                longest_pair = (name1, name2)

# Printing out the results
print("-------------------------------------------------------------------------------------")
print(f"Shortest baseline is between antennas {shortest_pair} with distance {shortest_baseline:.2f} meters")
print("-------------------------------------------------------------------------------------")
print(f"Longest baseline is between antennas {longest_pair} with distance {longest_baseline:.2f} meters")
print("-------------------------------------------------------------------------------------")

### 3) What is the expected resolution on the short and long baseline?


##### By definition, angular Resolution Calculation is given by
$$
\theta \approx \frac{\lambda}{B}
$$

So the one with the longest baseline will have the lowest resolution and similarly the one with the shortest baseline will have the highest resolution, so we get the following

In [ ]:
max_resolution = max(spectral_table[0]["RESOLUTION"].values)[0] #for the shortest baseline (shorter baseline yields a coarser resolution)
min_resolution = min(spectral_table[0]["RESOLUTION"].values)[0] #for the longest baseline (longer baseline leads to a higher resolution)

In [ ]:
print("-------------------------------------------------------------------------------------")
print(f"The expected resolution for the longest baseline is: {min_resolution}")
print("-------------------------------------------------------------------------------------")
print(f"The expected resolution for the shortest baseline is:{max_resolution}")
print("-------------------------------------------------------------------------------------")

In [ ]:
c= 3*10**(8)
Lambda = c/REF_FREQ

In [ ]:
Lambda

In [ ]:
RESOLUTION_low = Lambda/shortest_baseline
RESOLUTION_high = Lambda/longest_baseline

In [ ]:
RESOLUTION_low, RESOLUTION_high

### Calculate the delay that needs to be applied on the shortest and longest baseline if youare detecting a radio source in the sky.

#### Using that the delay time is given by:
$$
D = \frac{B \cdot \sin(\theta)}{c}
$$


In [ ]:
centroid_x = np.mean(X)
centroid_y = np.mean(Y)
centroid_z = np.mean(Z)

print("Centroid (X, Y, Z):", (centroid_x, centroid_y, centroid_z))

In [ ]:
# PHASE_DIR = np.array([[[5.1461782, -1.11199629]]])  
# frequency = REF_FREQ

# wavelength = c / frequency

# # Access the phase values
# phase_x = PHASE_DIR[0, 0, 0]  # First component (x)
# phase_y = PHASE_DIR[0, 0, 1]  # Second component (y)

# #
# phase_value = np.sqrt(phase_x**2 + phase_y**2)


# long_delay_time = (wavelength / (2 * np.pi) * phase_value)/longest_baseline
# short_delay_time = (wavelength / (2 * np.pi) * phase_value)/shortest_baseline

# print("Wavelength:", wavelength, "meters")
# print("Overall Phase:", phase_value)
# print("Long Delay Time:", long_delay_time, "seconds")
# print("Short Delay Time:", short_delay_time, "seconds")

In [ ]:
Field_table = xds_from_table(data_path+"1548939342.ms/::FIELD")
Field_table

In [ ]:
F_ID=Field_table[0]["DELAY_DIR"]
Phase_dir = Field_table[0]["PHASE_DIR"]
REF_DIR = Field_table[0]["REFERENCE_DIR"]

In [ ]:
# Given delay direction
delay_direction = np.array([[[5.1461782, -1.11199629]]])

# Extract x and y
x = delay_direction[0, 0, 0]
y = delay_direction[0, 0, 1]

# Calculate delay angle in radians
delay_angle_rad = np.arctan2(y, x)

# Convert delay angle to degrees
delay_angle_deg = np.degrees(delay_angle_rad)

delay_angle_rad, delay_angle_deg

#### Delay time for short baseline

In [ ]:
T_delay_short = np.abs((shortest_baseline*np.sin(delay_angle_rad))/c)

In [ ]:
f"{T_delay_short} seconds" 

#### Delay time for long baseline

In [ ]:
T_delay_long = np.abs((longest_baseline*np.sin(delay_angle_rad))/c)

In [ ]:
f"{T_delay_long} seconds"

### Detrending Signal

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.signal import detrend

# # Example radio signal with a linear trend (replace with actual data)
# t = np.linspace(0, 10, 1000)  # Time array
# signal_data = np.sin(2 * np.pi * t) + 0.5 * t  # Signal with a linear trend

# # Detrend the signal
# detrended_signal = detrend(signal_data)

# # Assume a flat baseline (zero line) for detrended signal
# expected_baseline = np.zeros_like(t)

# # Assuming noise uncertainty is constant (replace with actual uncertainties if known)
# sigma = np.std(detrended_signal)  # Standard deviation of detrended signal

# # Calculate chi-square (χ²)
# chi_squared = np.sum(((detrended_signal - expected_baseline) ** 2) / sigma ** 2)

# # Normalize χ² by the degrees of freedom (number of data points)
# chi_squared_normalized = chi_squared / len(detrended_signal)

# print(f"Chi-Square (χ²): {chi_squared}")
# print(f"Normalized Chi-Square: {chi_squared_normalized}")

# # Plot original and detrended signals for reference
# plt.figure(figsize=(10, 4))
# plt.plot(t, signal_data, label="Original Signal")
# plt.plot(t, detrended_signal, label="Detrended Signal", color='orange')
# plt.title("Original and Detrended Radio Signal")
# plt.xlabel("Time (s)")
# plt.ylabel("Amplitude")
# plt.legend()
# plt.show()

In [ ]:
AMMP= absdata
t = np.linspace(0, 100, len(absdata))  # Time array

In [ ]:
plt.plot(t, np.log(AMMP))
#plt.xlim(60,70)

In [ ]:
import numpy as np
from scipy.signal import detrend

Signal =  np.abs(vis[500,:,0])
Detrended_signal  = detrend(Signal)


plt.figure(figsize=(9,5.5))
plt.title("Detrending the signal",fontsize=  18)
plt.plot(Detrended_signal, label = "Detrended Signal")
plt.plot(Signal, label = "Signal")
plt.ylabel("Amplitude")
plt.xlabel("Channel")
plt.ylim(-20,40)
plt.savefig("Detrending_Signal.png")
plt.legend()


# Calculating the least squares residual (sum of squared differences)
residual = (np.sum((Signal - Detrended_signal )** 2))

print("Least squares residual:", residual.values)

### b. Explain how confident you are with your detrended signal?

### Redshift

Using the reference frequency and the frequency that we're focussed on which is 1.5GHz

By definition $z = \frac{f_{emit}}{f_{rest}}-1$

In [ ]:
Freq_1500MHz = 1.5*10**9 # In GHz
z = (Freq_1500MHz/REF_FREQ)-1

In [ ]:
print(f"The obtained redshift using the reference frequency is: {z}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt

# Assuming your signal is already defined
Signal = np.abs(vis[500, :, 0])  # This is your input signal

# Apply Median Filter to the signal
window_size = 3  # Adjust the window size based on your data
filtered_signal = medfilt(Signal, kernel_size=window_size)

# Calculate the removed RFI (difference between original and filtered signal)
rfi_removed = Signal - filtered_signal

# Plotting the original, filtered signals, and RFI removed
plt.figure(figsize=(12, 8))

# Original Signal
plt.subplot(3, 1, 1)
plt.plot(Signal, label='Original Signal (with potential RFI)', color='red')
plt.title('Original Signal')
plt.xlabel('Data Points')
plt.ylabel('Amplitude')
plt.legend()

# Filtered Signal
plt.subplot(3, 1, 2)
plt.plot(filtered_signal, label='Filtered Signal (RFI removed)', color='green')
plt.title('Filtered Signal after Median Filtering')
plt.xlabel('Data Points')
plt.ylabel('Amplitude')
plt.legend()

# RFI Removed (Difference between Original and Filtered)
plt.subplot(3, 1, 3)
plt.plot(rfi_removed, label='Detected RFI (Difference)', color='blue')
plt.title('RFI Removed from Signal')
plt.xlabel('Data Points')
plt.ylabel('Amplitude')
plt.legend()

plt.tight_layout()
plt.show()

### Locate any satellite/s that could have impacted that specific observation and calculate the angular separation of the satellite/s from your phase cente

In [ ]:
from skyfield.api import load, wgs84
from datetime import datetime
import numpy as np

# Load satellite data from CelesTrak
satellites = load.tle_file('http://celestrak.com/NORAD/elements/stations.txt')

#
# # Observer's coordinates
# observer_lat = -30.7133  # Latitude in decimal degrees
# observer_lon = 21.4431   # Longitude in decimal degrees
# observer_alt = 0         # Altitude in meters (optional)

# # Create observer location
# observer = wgs84.latlon(observer_lat, observer_lon, observer_alt)

# # Define the date of observation
# observation_date = datetime(2024, 10, 17)

# # Define the field of view in degrees (1.2 arcseconds converted to degrees)
# fov_radius = 1.2 / 3600  # 1.2 arcseconds to degrees

# # Load timescale
# ts = load.timescale()

# # Source coordinates (Right Ascension in hours, Declination in degrees)
# source_ra = 294.854277 / 15  # Right Ascension in hours
# source_dec = -63.712673  # Declination in degrees

# # Create a list to store visible satellites
# visible_satellites = []

# # Loop through the hours of the day to find visible satellites
# for hour in range(24):
#     observation_time = ts.utc(observation_date.year, observation_date.month, observation_date.day, hour)

#     # Check visibility for each satellite
#     for satellite in satellites:
#         # Get satellite's position at the observation time
#         satellite_position = satellite.at(observation_time)

#         # Calculate the satellite's apparent position from the observer
#         astrometric = satellite_position.observe(observer).apparent()

#         # Get apparent altitude and azimuth
#         alt, az, _ = astrometric.altaz()

#         # Check if the satellite is above the horizon
#         if alt.degrees > 0:
#             # Define the source position in the observer's sky
#             source_position = observer.at(observation_time).from_altaz(alt_degrees=source_dec, az_degrees=source_ra * 15)
            
#             # Calculate angular separation between the satellite and source
#             angular_distance = astrometric.separation_from(source_position).degrees

#             # If within the field of view, add to visible satellites
#             if angular_distance <= fov_radius:
#                 visible_satellites.append(satellite.name)

# # Remove duplicates and sort the results
# visible_satellites = list(set(visible_satellites))

# # Print the visible satellites
# print(f"Satellites visible in the field of view of {fov_radius * 3600} arcseconds around the source on {observation_date.strftime('%Y-%m-%d')}:")
# for satellite in visible_satellites:
#     print(satellite)

In [ ]:
from skyfield.api import load, Topos
from datetime import datetime
import numpy as np
from skyfield.api import wgs84

# Load satellite data
satellites = load.tle_file('http://celestrak.com/NORAD/elements/stations.txt')

# Observer's coordinates
observer_lat = -30.7133  # Latitude in decimal degrees
observer_lon = 21.4431   # Longitude in decimal degrees
observer_alt = 0         # Altitude in meters (optional)

# Create observer location
observer = Topos(latitude_degrees=observer_lat, longitude_degrees=observer_lon)

# Define the date of observation
start_date = datetime(2019 ,31, 1, 12, 56, 10.00)  # Example time 21:00:00
end_date = datetime(2019 ,31, 1, 12, 59, .00)  # Example time 21:00:00


# Define the field of view in degrees (1.2 arcseconds converted to degrees)
fov_radius = 1.2 / 3600  # 1.2 arcseconds to degrees

# Load the ephemeris to calculate positions
ts = load.timescale()
t = ts.utc(observation_date.year, observation_date.month, observation_date.day, observation_date.hour)

# Compute satellite positions and check if they are in the FoV
satellites_in_fov = []

for satellite in satellites:
    # Get the satellite's position at the observation time
    difference = satellite - observer
    topocentric = difference.at(t)
    
    # Get the alt-az coordinates of the satellite
    alt, az, distance = topocentric.altaz()
    
    # Check if the satellite is above the horizon and within the FoV radius
    if alt.degrees > 0 and np.abs(alt.degrees) < fov_radius:
        satellites_in_fov.append(satellite.name)

# Output the list of satellites in the field of view
print("Satellites in field of view:")
for sat in satellites_in_fov:
    print(sat)

In [ ]:
from skyfield.api import load, Topos
from datetime import datetime, timedelta
import numpy as np
import pytz

# Load satellite data
satellites = load.tle_file('http://celestrak.com/NORAD/elements/stations.txt')

# Observer's coordinates
observer_lat = -30.7133  # Latitude in decimal degrees
observer_lon = 21.4431   # Longitude in decimal degrees
observer_alt = 0         # Altitude in meters (optional)

# Create observer location
observer = Topos(latitude_degrees=observer_lat, longitude_degrees=observer_lon)

# Define the start and end times in MJD
start_mjd = 58514.53916802629
end_mjd = 58514.541389308404

# Convert MJD to datetime
def mjd_to_datetime(mjd):
    return datetime(1858, 11, 17) + timedelta(days=mjd)

# Convert start and end times to timezone-aware datetime
initial_date = pytz.utc.localize(mjd_to_datetime(start_mjd))
final_date = pytz.utc.localize(mjd_to_datetime(end_mjd))

# Define the field of view in degrees (1.2 arcseconds converted to degrees)
fov_radius = 1.2 / 3600  # 1.2 arcseconds to degrees

# Load timescale
ts = load.timescale()

# Initialize list to hold satellites in field of view
satellites_in_fov = []

# Check each satellite over the time range
current_date = initial_date
while current_date <= final_date:
    t = ts.utc(current_date)

    for sat in satellites:
        # Compute the position of the satellite and observer
        sat_pos = sat.at(t).position.au
        observer_pos = observer.at(t).position.au

        # Calculate the distance vector between the observer and the satellite
        d_vector = sat_pos - observer_pos

        # Calculate the distance from the observer to the satellite
        distance = np.linalg.norm(d_vector)

        # Calculate the angle using the dot product
        # Note: The observer's position is assumed to be at the origin in the local frame
        angle = np.arccos(np.dot(sat_pos, observer_pos) / (np.linalg.norm(sat_pos) * np.linalg.norm(observer_pos)))

        # Convert angle from radians to degrees
        angle_degrees = np.degrees(angle)

        # Check if the satellite is within the field of view
        if angle_degrees <= fov_radius:
            satellites_in_fov.append((sat, current_date))

    # Increment current date by a specific time step (e.g., 60 seconds)
    current_date += timedelta(seconds=60)

# Print out the satellites in the field of view with their MJD
print("Satellites in the field of view:")
for satellite, date in satellites_in_fov:
    mjd = (date - datetime(1858, 11, 17)).days
    print(f"{satellite.name} at MJD {mjd:.5f} on {date}")

In [16]:
from skyfield.api import load, Topos
from datetime import datetime, timedelta
import numpy as np

# Load satellite data
satellites = load.tle_file('http://celestrak.com/NORAD/elements/stations.txt')

# Observer's coordinates
observer_lat = -30.7133  # Latitude in decimal degrees
observer_lon = 21.4431   # Longitude in decimal degrees
observer_alt = 0         # Altitude in meters (optional)

# Create observer location
observer = Topos(latitude_degrees=observer_lat, longitude_degrees=observer_lon)

# Define the date of observation
start_date = datetime(2019, 1, 31, 12, 56, 10)  # Corrected date format
end_date = datetime(2019, 1, 31, 12, 59, 0)     # Corrected date format

# Define the field of view in degrees (1.2 arcseconds converted to degrees)
fov_radius = 61 / 60  # 1.2 arcseconds to degrees


source_ra = 294.854277 / 15  # Right Ascension in hours
source_dec = -63.712673  # Declination in degrees

ts = load.timescale()

# Compute satellite positions and check if they are in the FoV
satellites_in_fov = []

# Iterate over each second from start_date to end_date
current_time = start_date
while current_time <= end_date:
    t = ts.utc(current_time.year, current_time.month, current_time.day, 
               current_time.hour, current_time.minute, current_time.second)

    for satellite in satellites:
        # Get the satellite's position at the observation time
        difference = satellite - observer
        topocentric = difference.at(t)

        # Get the alt-az coordinates of the satellite
        alt, az, distance = topocentric.altaz()

        # Check if the satellite is above the horizon and within the FoV radius
        if alt.degrees > 0 and np.abs(az.degrees) < fov_radius:
            satellites_in_fov.append(satellite.name)

    # Move to the next second
    current_time += timedelta(seconds=1)

# Output the list of satellites in the field of view
print("Satellites in field of view:")
for sat in set(satellites_in_fov):  # Use set to avoid duplicates
    print(sat)

source_ra = 294.854277 / 15  # Right Ascension in hours
source_dec = -63.712673  # Declination in degrees

Satellites in field of view:
DORA
1998-067WL
1998-067WV
1998-067WY
BINAR-4
BINAR-2


In [22]:
from skyfield.api import load, Topos
from datetime import datetime, timedelta
import numpy as np

# Load satellite data
satellites = load.tle_file('http://celestrak.com/NORAD/elements/stations.txt')

# Observer's coordinates
observer_lat = -30.7133  # Latitude in decimal degrees
observer_lon = 21.4431   # Longitude in decimal degrees
observer_alt = 0         # Altitude in meters (optional)

# Create observer location
observer = Topos(latitude_degrees=observer_lat, longitude_degrees=observer_lon)

# Define the date of observation
start_date = datetime(2019, 1, 31, 12, 56, 10)
end_date = datetime(2019, 1, 31, 12, 59, 0)

# Field of view radius in degrees
fov_radius = 61 / 60  # 1.2 arcseconds to degrees

# Source coordinates
source_ra = 294.854277 / 15  # Right Ascension in hours
source_dec = -63.712673  # Declination in degrees

ts = load.timescale()

# Compute satellite positions and check if they are in the FoV
satellites_in_fov = []

# Iterate over each second from start_date to end_date
current_time = start_date
while current_time <= end_date:
    t = ts.utc(current_time.year, current_time.month, current_time.day, 
               current_time.hour, current_time.minute, current_time.second)

    for satellite in satellites:
        # Get the satellite's position at the observation time
        difference = satellite - observer
        topocentric = difference.at(t)

        # Get the alt-az coordinates of the satellite
        alt, az, distance = topocentric.altaz()

        # Check if the satellite is above the horizon and within the FoV radius
        if alt.degrees > 0:  # Check if the satellite is above the horizon
            # Check if within the field of view (assuming azimuth is needed for your context)
            if np.abs(az.degrees) < fov_radius:
                satellites_in_fov.append((satellite, t))

    # Move to the next second
    current_time += timedelta(seconds=1)

# Output the list of satellites in the field of view
print("Satellites in field of view:")
for sat, _ in set(satellites_in_fov):  # Use set to avoid duplicates
    print(sat)

# Calculate angular separation for each satellite in FoV
angular_separations = []
for satellite, t in satellites_in_fov:
    # Get the satellite's position at the observation time
    sat_pos = satellite.at(t).position.au  # Get position in AU
    sat_dec = np.degrees(np.arcsin(sat_pos[2] / np.linalg.norm(sat_pos)))  # Declination
    sat_ra = np.degrees(np.arctan2(sat_pos[1], sat_pos[0])) / 15  # Right Ascension in hours

    # Calculate angular separation
    delta_theta = np.degrees(np.arccos(
        np.sin(np.radians(source_dec)) * np.sin(np.radians(sat_dec)) +
        np.cos(np.radians(source_dec)) * np.cos(np.radians(sat_dec)) * 
        np.cos(np.radians(source_ra - sat_ra))
    ))

    print(f"Angular separation with {satellite.name}: {delta_theta:.2f} degrees")
    angular_separations.append(delta_theta)

print("All angular separations:", angular_separations)

Satellites in field of view:
BINAR-4 catalog #60952 epoch 2024-10-17 09:50:11 UTC
BINAR-2 catalog #60956 epoch 2024-10-17 09:04:00 UTC
1998-067WY catalog #60958 epoch 2024-10-17 09:30:44 UTC
BINAR-2 catalog #60956 epoch 2024-10-17 09:04:00 UTC
1998-067WL catalog #59561 epoch 2024-10-17 13:01:17 UTC
1998-067WV catalog #60955 epoch 2024-10-17 02:32:34 UTC
BINAR-4 catalog #60952 epoch 2024-10-17 09:50:11 UTC
DORA catalog #61502 epoch 2024-10-17 03:05:06 UTC
Angular separation with BINAR-2: 74.36 degrees
Angular separation with BINAR-2: 75.00 degrees
Angular separation with 1998-067WL: 53.55 degrees
Angular separation with DORA: 109.59 degrees
Angular separation with 1998-067WV: 76.36 degrees
Angular separation with BINAR-4: 103.90 degrees
Angular separation with 1998-067WY: 116.35 degrees
Angular separation with BINAR-4: 104.23 degrees
All angular separations: [74.36325538613104, 75.00184096003406, 53.552468118848324, 109.58889270450248, 76.35598101695334, 103.90273530503971, 116.34752680